<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/236_PredRevenue_Gap_Orchestrator_Tier2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Loading Utilities for Revenue Gap Orchestrator

In [ ]:
"""
Data Loading Utilities for Revenue Gap Orchestrator

Load and prepare data from CSV files.
"""

import pandas as pd
from pathlib import Path
from typing import Dict, List, Any, Optional


def load_sales_data(data_dir: Path) -> pd.DataFrame:
    """
    Load sales data from retail_weekly_sales.csv

    Args:
        data_dir: Path to data directory

    Returns:
        DataFrame with sales data
    """
    sales_file = data_dir / "retail_weekly_sales.csv"

    if not sales_file.exists():
        raise FileNotFoundError(f"Sales data file not found: {sales_file}")

    df = pd.read_csv(sales_file)

    # Convert date column to datetime
    df['week_start_date'] = pd.to_datetime(df['week_start_date'])

    return df


def load_stock_data(data_dir: Path) -> pd.DataFrame:
    """
    Load stock availability data from stock_availability.csv

    Args:
        data_dir: Path to data directory

    Returns:
        DataFrame with stock data
    """
    stock_file = data_dir / "stock_availability.csv"

    if not stock_file.exists():
        raise FileNotFoundError(f"Stock data file not found: {stock_file}")

    df = pd.read_csv(stock_file)

    # Convert date column to datetime
    df['week_start'] = pd.to_datetime(df['week_start'])

    return df


def load_customer_data(data_dir: Path) -> pd.DataFrame:
    """
    Load customer data from retail_customers.csv

    Args:
        data_dir: Path to data directory

    Returns:
        DataFrame with customer data
    """
    customer_file = data_dir / "retail_customers.csv"

    if not customer_file.exists():
        raise FileNotFoundError(f"Customer data file not found: {customer_file}")

    df = pd.read_csv(customer_file)

    return df


def build_sales_lookup(sales_df: pd.DataFrame) -> Dict[str, List[Dict[str, Any]]]:
    """
    Build lookup dictionary: customer_id -> list of sales records

    Args:
        sales_df: Sales DataFrame

    Returns:
        Dictionary mapping customer_id to list of sales records
    """
    lookup = {}

    for customer_id, group in sales_df.groupby('customer_id'):
        # Convert to list of dicts, handling datetime serialization
        records = []
        for _, row in group.iterrows():
            record = row.to_dict()
            # Convert datetime to string for JSON serialization
            if 'week_start_date' in record and pd.notna(record['week_start_date']):
                record['week_start_date'] = record['week_start_date'].strftime('%Y-%m-%d')
            # Convert numpy types to Python types
            for key, value in record.items():
                if pd.isna(value):
                    record[key] = None
                elif isinstance(value, (pd.Timestamp,)):
                    record[key] = value.strftime('%Y-%m-%d')
                elif hasattr(value, 'item'):  # numpy scalar
                    record[key] = value.item()
            records.append(record)

        lookup[str(customer_id)] = records

    return lookup


def filter_sales_by_customer(
    sales_df: pd.DataFrame,
    customer_id: Optional[str]
) -> pd.DataFrame:
    """
    Filter sales data by customer_id if provided

    Args:
        sales_df: Sales DataFrame
        customer_id: Customer ID to filter by (None = all customers)

    Returns:
        Filtered DataFrame
    """
    if customer_id is None:
        return sales_df

    return sales_df[sales_df['customer_id'] == int(customer_id)]


def merge_sales_stock(
    sales_df: pd.DataFrame,
    stock_df: pd.DataFrame
) -> pd.DataFrame:
    """
    Merge sales and stock data on store_id and date

    Args:
        sales_df: Sales DataFrame
        stock_df: Stock DataFrame

    Returns:
        Merged DataFrame with sales and stock information
    """
    # Merge on store_id and date
    merged = sales_df.merge(
        stock_df,
        left_on=['store_id', 'week_start_date'],
        right_on=['store_id', 'week_start'],
        how='left',
        suffixes=('_sales', '_stock')
    )

    return merged


def convert_dataframe_to_dict_list(df: pd.DataFrame) -> List[Dict[str, Any]]:
    """
    Convert DataFrame to list of dictionaries, handling special types

    Args:
        df: DataFrame to convert

    Returns:
        List of dictionaries
    """
    records = []

    for _, row in df.iterrows():
        record = {}
        for key, value in row.items():
            # Handle datetime
            if pd.isna(value):
                record[key] = None
            elif isinstance(value, pd.Timestamp):
                record[key] = value.strftime('%Y-%m-%d')
            elif hasattr(value, 'item'):  # numpy scalar
                record[key] = value.item()
            else:
                record[key] = value
        records.append(record)

    return records



# Data Loading Node

In [ ]:
def data_loading_node(state: PredictiveRevenueGapState) -> Dict[str, Any]:
    """
    Data Loading Node: Load sales, stock, and customer data.

    Orchestrates loading all required data sources.
    """
    errors = state.get("errors", [])
    customer_id = state.get("customer_id")
    data_dir_str = state.get("data_dir", "data")

    # Convert data_dir to Path
    data_dir = Path(data_dir_str)
    if not data_dir.is_absolute():
        # Relative to project root
        from pathlib import Path as P
        project_root = P(__file__).parent.parent.parent
        data_dir = project_root / data_dir_str

    try:
        # Load all data sources
        sales_df = load_sales_data(data_dir)
        stock_df = load_stock_data(data_dir)
        customers_df = load_customer_data(data_dir)

        # Filter sales by customer if specified
        if customer_id:
            sales_df = filter_sales_by_customer(sales_df, customer_id)
            # Also filter customer data
            customers_df = customers_df[customers_df['customer_id'] == int(customer_id)]

        # Build lookups
        sales_lookup = build_sales_lookup(sales_df)

        # Convert to dict lists for state
        sales_history = convert_dataframe_to_dict_list(sales_df)
        stock_data = convert_dataframe_to_dict_list(stock_df)
        all_customers = convert_dataframe_to_dict_list(customers_df)

        # If single customer, also store customer_data
        customer_data = None
        if customer_id and len(customers_df) > 0:
            customer_data = all_customers[0]

        return {
            "sales_history": sales_history,
            "stock_data": stock_data,
            "all_customers": all_customers,
            "sales_lookup": sales_lookup,
            "customer_data": customer_data,
            "errors": errors
        }

    except FileNotFoundError as e:
        return {
            "errors": errors + [f"data_loading_node: {str(e)}"]
        }
    except Exception as e:
        return {
            "errors": errors + [f"data_loading_node: Unexpected error - {str(e)}"]
        }


# Tests for Data Loading Utilities

In [ ]:
"""
Tests for Data Loading Utilities

Testing Phase 2: Data loading utilities before building the node
"""

import sys
from pathlib import Path
import pandas as pd

# Add project root to path
PROJECT_ROOT = Path(__file__).parent.parent
sys.path.insert(0, str(PROJECT_ROOT))

from agents.revenue_gap_orchestrator.utilities.data_loading import (
    load_sales_data,
    load_stock_data,
    load_customer_data,
    build_sales_lookup,
    filter_sales_by_customer,
    merge_sales_stock,
    convert_dataframe_to_dict_list
)


def test_load_sales_data():
    """Test loading sales data"""
    data_dir = PROJECT_ROOT / "data"

    df = load_sales_data(data_dir)

    assert isinstance(df, pd.DataFrame)
    assert len(df) > 0
    assert 'customer_id' in df.columns
    assert 'week_start_date' in df.columns
    assert 'weekly_spend' in df.columns
    assert 'store_id' in df.columns
    assert pd.api.types.is_datetime64_any_dtype(df['week_start_date'])
    print("✅ Load sales data test passed")


def test_load_stock_data():
    """Test loading stock data"""
    data_dir = PROJECT_ROOT / "data"

    df = load_stock_data(data_dir)

    assert isinstance(df, pd.DataFrame)
    assert len(df) > 0
    assert 'store_id' in df.columns
    assert 'sku' in df.columns
    assert 'week_start' in df.columns
    assert 'on_hand_units' in df.columns
    assert pd.api.types.is_datetime64_any_dtype(df['week_start'])
    print("✅ Load stock data test passed")


def test_load_customer_data():
    """Test loading customer data"""
    data_dir = PROJECT_ROOT / "data"

    df = load_customer_data(data_dir)

    assert isinstance(df, pd.DataFrame)
    assert len(df) > 0
    assert 'customer_id' in df.columns
    assert 'age' in df.columns
    assert 'household_size' in df.columns
    assert 'loyalty_member' in df.columns
    print("✅ Load customer data test passed")


def test_build_sales_lookup():
    """Test building sales lookup dictionary"""
    data_dir = PROJECT_ROOT / "data"
    sales_df = load_sales_data(data_dir)

    lookup = build_sales_lookup(sales_df)

    assert isinstance(lookup, dict)
    assert len(lookup) > 0

    # Check first customer
    first_customer_id = str(sales_df['customer_id'].iloc[0])
    assert first_customer_id in lookup
    assert isinstance(lookup[first_customer_id], list)
    assert len(lookup[first_customer_id]) > 0

    # Check record structure
    first_record = lookup[first_customer_id][0]
    assert 'customer_id' in first_record
    assert 'week_start_date' in first_record
    assert 'weekly_spend' in first_record
    print("✅ Build sales lookup test passed")


def test_filter_sales_by_customer():
    """Test filtering sales by customer_id"""
    data_dir = PROJECT_ROOT / "data"
    sales_df = load_sales_data(data_dir)

    # Test with specific customer
    customer_id = "1"
    filtered = filter_sales_by_customer(sales_df, customer_id)

    assert len(filtered) > 0
    assert all(filtered['customer_id'] == int(customer_id))

    # Test with None (all customers)
    all_customers = filter_sales_by_customer(sales_df, None)
    assert len(all_customers) == len(sales_df)
    print("✅ Filter sales by customer test passed")


def test_merge_sales_stock():
    """Test merging sales and stock data"""
    data_dir = PROJECT_ROOT / "data"
    sales_df = load_sales_data(data_dir)
    stock_df = load_stock_data(data_dir)

    merged = merge_sales_stock(sales_df, stock_df)

    assert isinstance(merged, pd.DataFrame)
    assert len(merged) > 0

    # Check that merged columns exist
    assert 'store_id' in merged.columns
    assert 'week_start_date' in merged.columns
    assert 'weekly_spend' in merged.columns
    assert 'on_hand_units' in merged.columns
    assert 'sku' in merged.columns

    # Check that some records have stock data (not all will match)
    assert merged['on_hand_units'].notna().any()
    print("✅ Merge sales stock test passed")


def test_convert_dataframe_to_dict_list():
    """Test converting DataFrame to list of dicts"""
    data_dir = PROJECT_ROOT / "data"
    sales_df = load_sales_data(data_dir)

    # Take a small sample
    sample_df = sales_df.head(5)
    records = convert_dataframe_to_dict_list(sample_df)

    assert isinstance(records, list)
    assert len(records) == 5

    # Check record structure
    first_record = records[0]
    assert isinstance(first_record, dict)
    assert 'customer_id' in first_record
    assert 'week_start_date' in first_record

    # Check date is string
    assert isinstance(first_record['week_start_date'], str)
    print("✅ Convert DataFrame to dict list test passed")


if __name__ == "__main__":
    print("Testing Data Loading Utilities...\n")

    test_load_sales_data()
    test_load_stock_data()
    test_load_customer_data()
    test_build_sales_lookup()
    test_filter_sales_by_customer()
    test_merge_sales_stock()
    test_convert_dataframe_to_dict_list()

    print("\n✅ All data loading utility tests passed!")



# Test Data Loading

In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_034_Predictive_Revenue_Gap_Orchestrator % python3 tests/test_data_loading_utilities.py
Testing Data Loading Utilities...

✅ Load sales data test passed
✅ Load stock data test passed
✅ Load customer data test passed
✅ Build sales lookup test passed
✅ Filter sales by customer test passed
✅ Merge sales stock test passed
✅ Convert DataFrame to dict list test passed

✅ All data loading utility tests passed!

# Revenue Analysis Utilities for Revenue Gap Orchestrator

# 🎯 Big Picture: What Revenue Analysis Is Doing

Revenue analysis establishes the **baseline logic** the orchestrator uses to judge:

1. **What is normal?** (baseline behavior)
2. **What changed?** (trend behavior)
3. **What will happen if we do nothing?** (prediction behavior)

This is foundational because ALL downstream nodes — gap detection, scoring, ranking — depend on these metrics being **correct, explainable, and stable**.

In orchestrators, consistency and predictability matter more than model complexity.

---

# 🧠 What You Should Be Learning & Understanding

Below is the “orchestrator-builder’s perspective” — not just what the code does, but **why** each part matters for multi-node workflows, error propagation, and interpretability.

---

# 1️⃣ **Baseline Calculation: Understanding “Normal”**

Function: `calculate_customer_baseline`

### What this is really doing:

It defines the “expected weekly revenue” for a customer over the baseline period.

This becomes your **control group** for comparison.

### What to pay attention to:

* Sorting by date — ensures consistent behavior across upstream nodes.
* Using **only the first N weeks** — creates a stable reference, not influenced by recent anomalies.
* Handles empty data (important in real pipelines).
* Uses raw customer data — no modeling, extremely reliable.

### Why this matters for orchestrators:

* Baselines must be **simple, deterministic, explainable**.
* If baselines fluctuate, your gap detection will be noisy.
* For enterprise systems, *explainability > complexity*.

If you understand this, you’re already thinking like a senior orchestrator architect.

---

# 2️⃣ **Trend Analysis: Understanding Change**

Function: `calculate_revenue_trend`

### What this is really doing:

Comparing early behavior (baseline) vs recent behavior to determine:

* Is behavior worsening?
* Improving?
* Stable?
* Do we have enough data to trust this signal?

### Key concepts to learn:

* **Trend stability** — protects the orchestrator from overreacting.
* **Recent vs baseline windows** — two competing forces:

  * Baseline: long-term normal
  * Recent: short-term change
* **Trend classification rules**:

  * < –15% → “declining”
  * > +15% → “growing”
  * else → “stable”

### Why this matters:

Trend classification is a **gatekeeper** for gap detection:

* If the trend is “stable,” many gap checks will never trigger.
* If the trend is “declining,” downstream rules will activate.

This is how orchestrators avoid false positives.

---

# 3️⃣ **Prediction: Projecting Risk Forward**

Function: `predict_revenue`

### What this is really doing:

Estimating future revenue to determine:

* Is a revenue gap temporary?
* Is decline accelerating?
* Should intervention be triggered?

### Concepts you should learn here:

#### **A. Moving Average**

The most stable short-term predictor — preferred when recent data is valid.

#### **B. Trend Projection**

Simple linear projection — used only when enough data exists.

#### **C. Baseline Fallback**

Important when customer is new or inconsistent.

#### **D. Confidence Score**

This is CRITICAL for orchestrators.

* It determines how much the orchestrator should trust its own prediction.
* Higher confidence → can escalate intervention.
* Lower confidence → orchestrator may avoid acting.

If you're building enterprise orchestrators, **confidence modeling** is essential.

---

# 4️⃣ **Multi-Customer Pipeline**

Function: `analyze_all_customers_revenue`

This shows how the orchestrator performs calculations at scale.

Learning objectives:

* **Loop over all entities (customers)**
* **Apply deterministic functions**
* **Merge results into a unified, machine-readable dictionary**
* **Keep enriched state consistent across nodes**

This builds your intuition for:

* Stateless vs stateful node design
* How utilities feed orchestrator nodes
* Batch processing patterns

---

# 🧩 What Makes This “Best-In-Class” Architecturally?

### ✔ Stability over complexity

You’re not training ML models yet — you're building **reliable signals**.

### ✔ Predictable feature engineering

Everything is deterministic and interpretable.

### ✔ Clear separation of concerns

Utilities do the math → nodes orchestrate decisions → LLM handles explanation.

### ✔ Allows LLMs to add value later

The structured output of these utilities is ideal for:

* Explanation
* Recommendation generation
* Summarization
* Debugging

### ✔ Perfect for automated decision-making

Your scoring node will rely on:

* baseline
* trend_percentage
* predicted_next_week
* predicted_next_month
* confidence

These are canonical risk-scoring signals.

---

# 🧱 Most Important Skills to Focus On (as an orchestrator builder)

Here’s what you want to master from this utility module:

---

## **Skill 1 — Temporal Windowing**

Choosing:

* how many weeks define a baseline
* how many weeks define a trend
* how far to predict

This affects sensitivity and robustness.

---

## **Skill 2 — Threshold Design**

The ±15% trend thresholds are business rules.

You should learn:

* how to make thresholds configurable
* how different thresholds change risk detection
* how to validate thresholds using real data

---

## **Skill 3 — Data Quality Handling**

Look at how the code:

* checks for insufficient data
* sorts data deterministically
* avoids division by zero
* returns safe defaults

This robustness is what makes a best-in-class orchestrator reliable.

---

## **Skill 4 — Combining Multiple Signals**

Revenue analysis produces **multiple independent features**:

* baseline avg
* recent avg
* trend %
* predictions
* confidence

These features will later power:

* gap detection
* scoring
* ranking

Learn how to treat each feature as:

* isolated
* explainable
* reusable

---

## **Skill 5 — Deterministic Logic**

Best orchestrators avoid “hidden randomness.”

Every utility is:

* repeatable
* deterministic
* auditable

This is essential for enterprise systems.

---

# 🎓 Summary: What You Should Take Away

Revenue analysis is not about ML — it's about building **strong, stable signals** and **interpretable metrics** that downstream nodes can rely on.

If you master:

* baseline logic
* trend logic
* prediction logic
* confidence modeling
* windowing
* threshold design
* deterministic processing

…you will develop the skillset required to build **enterprise-grade AI orchestrators** that are reliable, explainable, controllable, and safe.




In [ ]:
"""
Revenue Analysis Utilities for Revenue Gap Orchestrator

Calculate revenue baselines, trends, and predictions for customers.
"""

from typing import Dict, List, Any, Optional
from datetime import datetime, timedelta
import statistics


def calculate_customer_baseline(
    sales_records: List[Dict[str, Any]],
    baseline_weeks: int = 4
) -> Dict[str, Any]:
    """
    Calculate baseline revenue metrics for a customer.

    Args:
        sales_records: List of sales records for the customer (sorted by date)
        baseline_weeks: Number of weeks to use for baseline calculation

    Returns:
        Dictionary with baseline metrics
    """
    if not sales_records:
        return {
            "total_revenue": 0.0,
            "average_weekly_spend": 0.0,
            "weeks_active": 0,
            "baseline_weeks_avg": 0.0
        }

    # Sort by date to ensure correct order
    sorted_records = sorted(
        sales_records,
        key=lambda x: x.get('week_start_date', '')
    )

    # Calculate total revenue
    total_revenue = sum(record.get('weekly_spend', 0.0) for record in sorted_records)
    weeks_active = len(sorted_records)
    average_weekly_spend = total_revenue / weeks_active if weeks_active > 0 else 0.0

    # Calculate baseline (first N weeks)
    baseline_records = sorted_records[:baseline_weeks]
    baseline_revenue = sum(record.get('weekly_spend', 0.0) for record in baseline_records)
    baseline_weeks_avg = baseline_revenue / len(baseline_records) if baseline_records else 0.0

    return {
        "total_revenue": round(total_revenue, 2),
        "average_weekly_spend": round(average_weekly_spend, 2),
        "weeks_active": weeks_active,
        "baseline_weeks_avg": round(baseline_weeks_avg, 2)
    }


def calculate_revenue_trend(
    sales_records: List[Dict[str, Any]],
    baseline_weeks: int = 4,
    recent_weeks: int = 4
) -> Dict[str, Any]:
    """
    Calculate revenue trend (declining/stable/growing).

    Args:
        sales_records: List of sales records for the customer (sorted by date)
        baseline_weeks: Number of weeks for baseline period
        recent_weeks: Number of recent weeks for comparison

    Returns:
        Dictionary with trend analysis
    """
    if not sales_records or len(sales_records) < max(baseline_weeks, recent_weeks):
        return {
            "revenue_trend": "insufficient_data",
            "recent_weeks_avg": 0.0,
            "baseline_weeks_avg": 0.0,
            "trend_percentage": 0.0
        }

    # Sort by date
    sorted_records = sorted(
        sales_records,
        key=lambda x: x.get('week_start_date', '')
    )

    # Baseline period (first N weeks)
    baseline_records = sorted_records[:baseline_weeks]
    baseline_avg = sum(r.get('weekly_spend', 0.0) for r in baseline_records) / len(baseline_records)

    # Recent period (last N weeks)
    recent_records = sorted_records[-recent_weeks:]
    recent_avg = sum(r.get('weekly_spend', 0.0) for r in recent_records) / len(recent_records)

    # Calculate trend percentage
    if baseline_avg > 0:
        trend_percentage = ((recent_avg - baseline_avg) / baseline_avg) * 100
    else:
        trend_percentage = 0.0

    # Classify trend
    if trend_percentage < -15.0:
        trend = "declining"
    elif trend_percentage > 15.0:
        trend = "growing"
    else:
        trend = "stable"

    return {
        "revenue_trend": trend,
        "recent_weeks_avg": round(recent_avg, 2),
        "baseline_weeks_avg": round(baseline_avg, 2),
        "trend_percentage": round(trend_percentage, 2)
    }


def predict_revenue(
    sales_records: List[Dict[str, Any]],
    prediction_horizon_weeks: int = 4,
    baseline_weeks: int = 4,
    recent_weeks: int = 4
) -> Dict[str, Any]:
    """
    Predict future revenue using simple methods.

    Args:
        sales_records: List of sales records for the customer
        prediction_horizon_weeks: Weeks ahead to predict
        baseline_weeks: Weeks for baseline calculation
        recent_weeks: Weeks for recent average

    Returns:
        Dictionary with predictions
    """
    if not sales_records:
        return {
            "predicted_next_week": 0.0,
            "predicted_next_month": 0.0,
            "prediction_method": "no_data",
            "confidence": 0.0
        }

    # Sort by date
    sorted_records = sorted(
        sales_records,
        key=lambda x: x.get('week_start_date', '')
    )

    # Method 1: Moving average (recent weeks)
    recent_records = sorted_records[-recent_weeks:]
    moving_avg = sum(r.get('weekly_spend', 0.0) for r in recent_records) / len(recent_records)

    # Method 2: Trend projection
    baseline_records = sorted_records[:baseline_weeks]
    baseline_avg = sum(r.get('weekly_spend', 0.0) for r in baseline_records) / len(baseline_records)
    recent_avg = sum(r.get('weekly_spend', 0.0) for r in recent_records) / len(recent_records)

    trend_per_week = (recent_avg - baseline_avg) / max(len(sorted_records), 1)
    trend_projection = recent_avg + (trend_per_week * prediction_horizon_weeks)

    # Method 3: Baseline (fallback)
    baseline_prediction = baseline_avg

    # Choose prediction method based on data quality
    if len(sorted_records) >= recent_weeks:
        # Use moving average if we have enough recent data
        predicted_next_week = moving_avg
        predicted_next_month = moving_avg * prediction_horizon_weeks
        prediction_method = "moving_average"
        confidence = min(0.9, len(sorted_records) / 12.0)  # Higher confidence with more data
    elif len(sorted_records) >= baseline_weeks:
        # Use baseline if limited data
        predicted_next_week = baseline_prediction
        predicted_next_month = baseline_prediction * prediction_horizon_weeks
        prediction_method = "baseline"
        confidence = 0.6
    else:
        # Insufficient data
        predicted_next_week = 0.0
        predicted_next_month = 0.0
        prediction_method = "insufficient_data"
        confidence = 0.0

    return {
        "predicted_next_week": round(predicted_next_week, 2),
        "predicted_next_month": round(predicted_next_month, 2),
        "prediction_method": prediction_method,
        "confidence": round(confidence, 2)
    }


def analyze_all_customers_revenue(
    sales_lookup: Dict[str, List[Dict[str, Any]]],
    baseline_weeks: int = 4,
    recent_weeks: int = 4,
    prediction_horizon_weeks: int = 4
) -> Dict[str, Dict[str, Any]]:
    """
    Analyze revenue for all customers.

    Args:
        sales_lookup: Dictionary mapping customer_id to sales records
        baseline_weeks: Weeks for baseline calculation
        recent_weeks: Weeks for recent trend analysis
        prediction_horizon_weeks: Weeks ahead to predict

    Returns:
        Dictionary mapping customer_id to revenue analysis
    """
    customer_baselines = {}

    for customer_id, sales_records in sales_lookup.items():
        # Calculate baseline
        baseline = calculate_customer_baseline(sales_records, baseline_weeks)

        # Calculate trend
        trend = calculate_revenue_trend(sales_records, baseline_weeks, recent_weeks)

        # Predict revenue
        prediction = predict_revenue(
            sales_records,
            prediction_horizon_weeks,
            baseline_weeks,
            recent_weeks
        )

        # Combine all metrics
        customer_baselines[customer_id] = {
            "customer_id": customer_id,
            **baseline,
            **trend,
            **prediction
        }

    return customer_baselines



# Tests for Revenue Analysis Utilities

In [ ]:
"""
Tests for Revenue Analysis Utilities

Testing Phase 3: Revenue analysis utilities before building the node
"""

import sys
from pathlib import Path

# Add project root to path
PROJECT_ROOT = Path(__file__).parent.parent
sys.path.insert(0, str(PROJECT_ROOT))

from agents.revenue_gap_orchestrator.utilities.revenue_analysis import (
    calculate_customer_baseline,
    calculate_revenue_trend,
    predict_revenue,
    analyze_all_customers_revenue
)


def test_calculate_customer_baseline():
    """Test baseline calculation"""
    sales_records = [
        {"week_start_date": "2025-09-06", "weekly_spend": 50.0},
        {"week_start_date": "2025-09-13", "weekly_spend": 45.0},
        {"week_start_date": "2025-09-20", "weekly_spend": 55.0},
        {"week_start_date": "2025-09-27", "weekly_spend": 40.0},
    ]

    baseline = calculate_customer_baseline(sales_records, baseline_weeks=4)

    assert baseline["total_revenue"] == 190.0
    assert baseline["weeks_active"] == 4
    assert baseline["average_weekly_spend"] == 47.5
    assert baseline["baseline_weeks_avg"] == 47.5
    print("✅ Calculate customer baseline test passed")


def test_calculate_customer_baseline_empty():
    """Test baseline calculation with empty records"""
    baseline = calculate_customer_baseline([], baseline_weeks=4)

    assert baseline["total_revenue"] == 0.0
    assert baseline["weeks_active"] == 0
    assert baseline["average_weekly_spend"] == 0.0
    print("✅ Calculate customer baseline (empty) test passed")


def test_calculate_revenue_trend_declining():
    """Test trend calculation for declining revenue"""
    sales_records = [
        {"week_start_date": "2025-09-06", "weekly_spend": 50.0},  # Baseline
        {"week_start_date": "2025-09-13", "weekly_spend": 48.0},  # Baseline
        {"week_start_date": "2025-09-20", "weekly_spend": 46.0},  # Baseline
        {"week_start_date": "2025-09-27", "weekly_spend": 44.0},  # Baseline
        {"week_start_date": "2025-10-04", "weekly_spend": 30.0},  # Recent
        {"week_start_date": "2025-10-11", "weekly_spend": 28.0},  # Recent
        {"week_start_date": "2025-10-18", "weekly_spend": 26.0},  # Recent
        {"week_start_date": "2025-10-25", "weekly_spend": 24.0},  # Recent
    ]

    trend = calculate_revenue_trend(sales_records, baseline_weeks=4, recent_weeks=4)

    assert trend["revenue_trend"] == "declining"
    assert trend["trend_percentage"] < -15.0
    print("✅ Calculate revenue trend (declining) test passed")


def test_calculate_revenue_trend_growing():
    """Test trend calculation for growing revenue"""
    sales_records = [
        {"week_start_date": "2025-09-06", "weekly_spend": 30.0},  # Baseline
        {"week_start_date": "2025-09-13", "weekly_spend": 32.0},  # Baseline
        {"week_start_date": "2025-09-20", "weekly_spend": 34.0},  # Baseline
        {"week_start_date": "2025-09-27", "weekly_spend": 36.0},  # Baseline
        {"week_start_date": "2025-10-04", "weekly_spend": 50.0},  # Recent
        {"week_start_date": "2025-10-11", "weekly_spend": 52.0},  # Recent
        {"week_start_date": "2025-10-18", "weekly_spend": 54.0},  # Recent
        {"week_start_date": "2025-10-25", "weekly_spend": 56.0},  # Recent
    ]

    trend = calculate_revenue_trend(sales_records, baseline_weeks=4, recent_weeks=4)

    assert trend["revenue_trend"] == "growing"
    assert trend["trend_percentage"] > 15.0
    print("✅ Calculate revenue trend (growing) test passed")


def test_calculate_revenue_trend_stable():
    """Test trend calculation for stable revenue"""
    sales_records = [
        {"week_start_date": "2025-09-06", "weekly_spend": 50.0},
        {"week_start_date": "2025-09-13", "weekly_spend": 48.0},
        {"week_start_date": "2025-09-20", "weekly_spend": 52.0},
        {"week_start_date": "2025-09-27", "weekly_spend": 50.0},
        {"week_start_date": "2025-10-04", "weekly_spend": 49.0},
        {"week_start_date": "2025-10-11", "weekly_spend": 51.0},
        {"week_start_date": "2025-10-18", "weekly_spend": 50.0},
        {"week_start_date": "2025-10-25", "weekly_spend": 52.0},
    ]

    trend = calculate_revenue_trend(sales_records, baseline_weeks=4, recent_weeks=4)

    assert trend["revenue_trend"] == "stable"
    assert -15.0 <= trend["trend_percentage"] <= 15.0
    print("✅ Calculate revenue trend (stable) test passed")


def test_predict_revenue():
    """Test revenue prediction"""
    sales_records = [
        {"week_start_date": "2025-09-06", "weekly_spend": 50.0},
        {"week_start_date": "2025-09-13", "weekly_spend": 48.0},
        {"week_start_date": "2025-09-20", "weekly_spend": 52.0},
        {"week_start_date": "2025-09-27", "weekly_spend": 50.0},
        {"week_start_date": "2025-10-04", "weekly_spend": 49.0},
        {"week_start_date": "2025-10-11", "weekly_spend": 51.0},
        {"week_start_date": "2025-10-18", "weekly_spend": 50.0},
        {"week_start_date": "2025-10-25", "weekly_spend": 52.0},
    ]

    prediction = predict_revenue(sales_records, prediction_horizon_weeks=4)

    assert "predicted_next_week" in prediction
    assert "predicted_next_month" in prediction
    assert "prediction_method" in prediction
    assert "confidence" in prediction
    assert prediction["predicted_next_week"] > 0
    assert prediction["predicted_next_month"] > 0
    print("✅ Predict revenue test passed")


def test_analyze_all_customers_revenue():
    """Test analyzing all customers"""
    sales_lookup = {
        "1": [
            {"week_start_date": "2025-09-06", "weekly_spend": 50.0},
            {"week_start_date": "2025-09-13", "weekly_spend": 45.0},
            {"week_start_date": "2025-09-20", "weekly_spend": 55.0},
            {"week_start_date": "2025-09-27", "weekly_spend": 40.0},
        ],
        "2": [
            {"week_start_date": "2025-09-06", "weekly_spend": 100.0},
            {"week_start_date": "2025-09-13", "weekly_spend": 95.0},
            {"week_start_date": "2025-09-20", "weekly_spend": 105.0},
            {"week_start_date": "2025-09-27", "weekly_spend": 100.0},
        ]
    }

    results = analyze_all_customers_revenue(sales_lookup, baseline_weeks=4, recent_weeks=4)

    assert len(results) == 2
    assert "1" in results
    assert "2" in results

    # Check structure
    customer_1 = results["1"]
    assert "customer_id" in customer_1
    assert "total_revenue" in customer_1
    assert "revenue_trend" in customer_1
    assert "predicted_next_week" in customer_1
    print("✅ Analyze all customers revenue test passed")


if __name__ == "__main__":
    print("Testing Revenue Analysis Utilities...\n")

    test_calculate_customer_baseline()
    test_calculate_customer_baseline_empty()
    test_calculate_revenue_trend_declining()
    test_calculate_revenue_trend_growing()
    test_calculate_revenue_trend_stable()
    test_predict_revenue()
    test_analyze_all_customers_revenue()

    print("\n✅ All revenue analysis utility tests passed!")



# Test Revenue Utils

In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_034_Predictive_Revenue_Gap_Orchestrator % python3 tests/test_revenue_analysis_utilities.py
Testing Revenue Analysis Utilities...

✅ Calculate customer baseline test passed
✅ Calculate customer baseline (empty) test passed
✅ Calculate revenue trend (declining) test passed
✅ Calculate revenue trend (growing) test passed
✅ Calculate revenue trend (stable) test passed
✅ Predict revenue test passed
✅ Analyze all customers revenue test passed

✅ All revenue analysis utility tests passed!


Your current orchestrator framework is **both fully functional as-is** *and* intentionally designed to be **extremely flexible** so that you can plug in more sophisticated ML components later without rewriting anything.

---

# ✅ **1. Current Framework: Simple, Interpretable, Reliable**

The current revenue analysis utilities (baseline, trend, prediction) are intentionally:

* Deterministic
* Explainable
* Lightweight
* Fast
* Easy to audit

They use **rolling windows**, not ML models, because:

* MVPs need interpretability
* It avoids overfitting
* It avoids needing a training pipeline
* It keeps the orchestrator focused on *workflow logic*, not ML training

This is how *actual enterprise orchestrator MVPs* are built.

---

# 🚀 **2. Adding ML? You simply plug it in as another utility module**

Yes — if a company wants ML predictions, you **add one new utility** and **one new node**, without removing or breaking anything.

For example:

```
utilities/
  revenue_analysis.py
  revenue_ml_prediction.py   ← NEW
nodes/
  revenue_analysis_node.py
  revenue_ml_prediction_node.py  ← NEW
```

This new module might include:

* forecast models (Prophet, ARIMA, XGBoost, LSTM, Transformers)
* probability-of-churn models
* customer lifetime value (CLV) models
* uplift models
* anomaly detection models

Then in the orchestrator flow:

### Current Flow:

```
plan → data load → revenue analysis → gap detection → scoring → ranking → report
```

### With ML Added:

```
plan → data load → revenue analysis → ML prediction → gap detection → scoring → ranking → report
```

ML predictions simply *augment* the state.

---

# 🧠 **3. The State Architecture Already Supports ML Outputs**

Your `PredictiveRevenueGapState` (from your STATE_DESIGN doc) already contains patterns like:

* `baseline_weeks`
* `recent_weeks`
* `predicted_next_week`
* `predicted_next_month`
* `confidence`

You can easily extend it with ML fields:

* `ml_predicted_next_week`
* `ml_predicted_monthly_revenue`
* `ml_churn_probability`
* `ml_gap_probability`
* `ml_confidence_score`

No breaking changes.
Just add keys.

---

# 🧩 **4. Why your architecture is ideal for ML add-ons**

Your orchestrator is:

### 🔹 Node-based

Each step is independent — perfect for swapping in ML logic.

### 🔹 State-driven

All intermediate computations live in the state, so new model outputs are easy to store.

### 🔹 Modular utilities

ML logic can live in its own file without touching existing utilities.

### 🔹 Workflow-sequenced

ML predictions can be placed anywhere in the sequence:

* before gap detection
* before scoring
* as the final scoring mechanism
* or as the ranking engine

### 🔹 LLM-friendly

You can even use:

* LLMs for explanation
* ML for prediction
* Stock data for reasoning
  All at once.

---

# 🏗️ **5. How ML is added (simple example)**

### New utility (e.g., XGBoost)

```python
def ml_predict_revenue(customer_features: Dict[str, Any], model):
    prediction = model.predict([customer_features])
    return {
        "ml_predicted_next_week": float(prediction[0]),
        "ml_confidence_score": 0.85
    }
```

### New node

```python
def revenue_ml_prediction_node(state):
    features = extract_ml_features(state.sales_history)
    ml_results = ml_predict_revenue(features, state.ml_model)
    state.update(ml_results)
    return state
```

That's it.

---

# 🎯 **6. Bottom Line**

### ✔ Your current framework is MVP-ready

### ✔ It is fully functional with interpretable revenue logic

### ✔ It is designed for scalability

### ✔ ML can be added later through a **completely clean extension path**

### ✔ No code rewrites are needed

### ✔ Best practice orchestration architecture used in enterprise AI

